In [102]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [73]:
train = pd.read_csv('Titanic/train.csv')
test = pd.read_csv('Titanic/test.csv')
gend = pd.read_csv('Titanic/gender_submission.csv')
# print(gend.head())
# print(test.head())

dftest = test.copy()

print(test.describe())
print(test.isnull().sum())



       PassengerId      Pclass         Age       SibSp       Parch        Fare
count   418.000000  418.000000  332.000000  418.000000  418.000000  417.000000
mean   1100.500000    2.265550   30.272590    0.447368    0.392344   35.627188
std     120.810458    0.841838   14.181209    0.896760    0.981429   55.907576
min     892.000000    1.000000    0.170000    0.000000    0.000000    0.000000
25%     996.250000    1.000000   21.000000    0.000000    0.000000    7.895800
50%    1100.500000    3.000000   27.000000    0.000000    0.000000   14.454200
75%    1204.750000    3.000000   39.000000    1.000000    0.000000   31.500000
max    1309.000000    3.000000   76.000000    8.000000    9.000000  512.329200
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [91]:
median_age = train['Age'].median()
dftest['Age'] = test['Age'].fillna(median_age)
print(dftest.isnull().sum())

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
